In [74]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [75]:
class FaceoffsDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.data.iloc[idx, 1:]
        data = np.array([data])
        data = data.reshape(-1, 2)

        if self.transform:
            data = self.transform(data)

        return data

In [76]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")

In [77]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
print(data.head(100))
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.65, test_size=0.35, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.571, random_state=42)
# Above two lines in unison accomplish a 65-15-20 split for train-test-validation

X_train = torch.from_numpy(X_train.to_numpy())
X_val = torch.from_numpy(X_val.to_numpy())
X_test = torch.from_numpy(X_test.to_numpy())

y_train = torch.from_numpy(y_train.to_numpy())
y_val = torch.from_numpy(y_val.to_numpy())
y_test = torch.from_numpy(y_test.to_numpy())

print(X_train.shape)

    game_id_x  season_x  game_seconds  event_index  game_period  coords_x  \
0   -1.798257 -1.797913     -0.715929    -0.712210    -0.606279 -0.531820   
1   -1.798257 -1.797913     -0.697547    -0.677103    -0.606279 -0.531820   
2   -1.798257 -1.797913      0.039564     0.060154    -0.606279 -0.531820   
3   -1.798257 -1.797913      0.210515     0.255753    -0.606279  1.880391   
4   -1.798257 -1.797913      0.445803     0.471414     0.764344 -0.531820   
..        ...       ...           ...          ...          ...       ...   
95  -1.798252 -1.797913     -0.357483    -0.326028    -0.606279  1.880391   
96  -1.798252 -1.797913      0.458670     0.476429     0.764344  1.880391   
97  -1.798252 -1.797913      0.554256     0.556675     0.764344  1.880391   
98  -1.798252 -1.797913      0.604806     0.596797     0.764344 -0.531820   
99  -1.798252 -1.797913      0.816197     0.772335     0.764344  1.880391   

    coords_y  home_score  away_score  event_distance  ...  \
0  -0.532003  

In [78]:
class VanillaNeuralNet(nn.Module):

    def __init__(self):
        super(VanillaNeuralNet, self).__init__()
        self.fc1 = nn.Linear(1500, 1000)
        self.fc2 = nn.Linear(1000, 750)
        self.fc3 = nn.Linear(750, 500)
        self.fc4 = nn.Linear(500, 500)
        self.fc5 = nn.Linear(500, 500)
        self.fc6 = nn.Linear(500, 500)
        self.fc7 = nn.Linear(500, 250)
        self.fc8 = nn.Linear(250, 100)
        self.fc9 = nn.Linear(100, 50)
        self.fc10 = nn.Linear(50, 1) # output singular prediction
    
    def forward(self, x):
        x_1 = self.fc1(torch.nn.functional.relu(x))
        x_2 = self.fc2(torch.nn.functional.relu(x_1))
        x_3 = self.fc3(torch.nn.functional.relu(x_2))
        x_4 = self.fc4(torch.nn.functional.relu(x_3))
        x_5 = self.fc5(torch.nn.functional.relu(x_4))
        x_6 = self.fc6(torch.nn.functional.relu(x_5))
        x_7 = self.fc7(torch.nn.functional.relu(x_6))
        x_8 = self.fc8(torch.nn.functional.relu(x_7))
        x_9 = self.fc9(torch.nn.functional.relu(x_8))
        x_10 = self.fc10(torch.nn.functional.relu(x_9))
        return x_10

In [79]:
class CreateDataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    
    def __getitem__(self, index):
        sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32), 
                    'label': torch.tensor(self.y[index], dtype=torch.float32)}

        return sample


    def __len__(self):
        return len(self.x)

In [80]:
training_dataset = CreateDataset(X_train, y_train)
validation_dataset = CreateDataset(X_val, y_val)
print(training_dataset.x.shape)
print(training_dataset.y.shape)
print(validation_dataset)

torch.Size([60387, 529])
torch.Size([60387])


In [81]:
import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
        torch.cuda.empty_cache()

model = VanillaNeuralNet().to(device)
learning_rate = 0.01
train_batch_size = 300
validation_batch_size = 300
train_dataloader = DataLoader(training_dataset, batch_size=train_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=validation_batch_size, shuffle=False)
## Initialize Optimizer and Learning Rate Scheduler
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()
print("Start Training...")
num_epochs = 200
training_losses = np.array([])
validation_losses = np.array([])
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_dataloader):
      X = X.to(device)
      y = y.to(device)
      prediction = model(X).to(device)
      optimizer.zero_grad()
      loss = loss_func(prediction, y)
      loss.backward()
      optimizer.step()
      train_loss += loss.float().item()
      count += 1
    scheduler.step()
    training_losses = np.append(training_losses, train_loss / count)
    print("Training loss:", train_loss / count)
    model.train(False)

    ########################### Validation #####################################
    # TODO: Design your own validation section

    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
      for X, y in validation_dataloader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y)
        val_loss += loss.float().item()
        count += 1
    print("Validation loss:", val_loss / count)
    validation_losses = np.append(validation_losses, val_loss / count)

torch.save(model, "colorization_model_1.pt")


Start Training...

EPOCH 1 of 200



C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32),
C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(self.y[index], dtype=torch.float32)}


AttributeError: 'str' object has no attribute 'to'

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
        torch.cuda.empty_cache()

model = VanillaNeuralNet().to(device)
learning_rate = 0.01
train_batch_size = 300
validation_batch_size = 300
train_dataloader = DataLoader(training_dataset, batch_size=train_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=validation_batch_size, shuffle=False)
## Initialize Optimizer and Learning Rate Scheduler
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

In [64]:
print("Start Training...")
num_epochs = 200
training_losses = np.array([])
validation_losses = np.array([])
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_dataloader):
      X = X.to(device)
      y = y.to(device)
      prediction = model(X).to(device)
      optimizer.zero_grad()
      loss = loss_func(prediction, y)
      loss.backward()
      optimizer.step()
      train_loss += loss.float().item()
      count += 1
    scheduler.step()
    training_losses = np.append(training_losses, train_loss / count)
    print("Training loss:", train_loss / count)
    model.train(False)

    ########################### Validation #####################################
    # TODO: Design your own validation section

    model.eval()
    val_loss = 0
    count = 0
    with torch.no_grad():
      for X, y in validation_dataloader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        loss = loss_func(prediction, y)
        val_loss += loss.float().item()
        count += 1
    print("Validation loss:", val_loss / count)
    validation_losses = np.append(validation_losses, val_loss / count)

torch.save(model, "colorization_model_1.pt")

Start Training...

EPOCH 1 of 200



C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'feature': torch.tensor(self.x[index], dtype=torch.float32),
C:\Users\Tad\AppData\Local\Temp\ipykernel_13644\4281265504.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(self.y[index], dtype=torch.float32)}


AttributeError: 'str' object has no attribute 'to'